In [12]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
import os
import pickle

model_pth = "/home/gs/aigc/modelzoo"
model_name = {
    "openpose": os.path.join(model_pth, "openpose.pkl"),
    "ctl_openpose": os.path.join(model_pth, "sd-controlnet-openpose"),
    "ctl_canny": os.path.join(model_pth, "sd-controlnet-canny"),
    "sd": os.path.join(model_pth, "stable-diffusion-v1-5"),
}

if os.path.exists(model_name["openpose"]):
    with open(model_name["openpose"], "rb") as fo:
        openpose = pickle.load(fo)
else:
    openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
    with open(model_name["openpose"], "wb") as fo:
        pickle.dump(openpose, fo)
ctl_openpose = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")
# ctl_openpose.save_pretrained(model_name["ctl_openpose"])

ctl_canny = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny")
# ctl_canny.save_pretrained(model_name["ctl_canny"])
controlnet = [
    ctl_openpose,
    ctl_canny
]
pipe = StableDiffusionControlNetPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", controlnet=controlnet)
pipe.save_pretrained(model_name["sd"])
print(f"*** {model_name['sd']} saved")



Loading pipeline components...:   0%|                                                                                    | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.45it/s]


*** /home/gs/aigc/modelzoo/stable-diffusion-v1-5 saved


In [10]:
import diffusers
diffusers.__version__

'0.20.0.dev0'

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
from PIL import Image
import cv2
import numpy as np
from controlnet_aux import OpenposeDetector
import torch

def canny_extractor(img_file):
    canny_image = load_image(img_file)
    canny_image = np.array(canny_image)
    
    low_threshold = 100
    high_threshold = 200
    
    canny_image = cv2.Canny(canny_image, low_threshold, high_threshold)
    
    # zero out middle columns of image where pose will be overlayed
    zero_start = canny_image.shape[1] // 4
    zero_end = zero_start + canny_image.shape[1] // 2
    canny_image[:, zero_start:zero_end] = 0
    
    canny_image = canny_image[:, :, None]
    canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
    canny_image = Image.fromarray(canny_image)

    return canny_image

def openpose_extractor(img_file, openpose_model):
    if os.path.exists(openpose_model):
        with open(openpose_model, "rb") as fo:
            openpose = pickle.load(fo)
    else:
        openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
        with open(openpose_model, "wb") as fo:
            pickle.dump(openpose, fo)
    i_image = load_image(img_file)
    openpose_image = openpose(i_image)

    return openpose_image


model_pth = "/home/gs/aigc/modelzoo"
model_name = {
    "openpose": os.path.join(model_pth, "controlnet/openpose.pkl"),
    "ctl_openpose": os.path.join(model_pth, "controlnet/sd-controlnet-openpose"),
    "ctl_canny": os.path.join(model_pth, "controlnet/sd-controlnet-canny"),
    "sd": os.path.join(model_pth, "stable-diffusion-v1-5"),
}

is_gpu = torch.cuda.is_available()
if is_gpu:
    device = "cuda"
    md_dtype = torch.float16
else:
    device = "cpu"
    md_dtype = torch.float32

canny_image = canny_extractor(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/landscape.png"
)

openpose_image = openpose_extractor(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/person.png",
    model_name["openpose"]
)

controlnet = [
    ControlNetModel.from_pretrained(model_name["ctl_openpose"], torch_dtype=md_dtype),
    ControlNetModel.from_pretrained(model_name["ctl_canny"], torch_dtype=md_dtype),
]

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    model_name["sd"], controlnet=controlnet, torch_dtype=md_dtype
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

if is_gpu:
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_model_cpu_offload()

prompt = "a giant standing in a fantasy landscape, best quality"
negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality"

generator = torch.Generator(device="cpu").manual_seed(1)

images = [openpose_image, canny_image]

image = pipe(
    prompt,
    images,
    num_inference_steps=2,
    generator=generator,
    negative_prompt=negative_prompt,
    controlnet_conditioning_scale=[1.0, 0.8],
).images[0]

image.save("./multi_controlnet_output.png")
